In [154]:
import urllib2
from bs4 import BeautifulSoup
import re
import nytimesarticle as nyta
import urllib2
import requests
import csv
import time
import pandas as pd
import numpy as np
import nytimes_crawl_2 as nytc

In [159]:
%load_ext autoreload
%autoreload 2

# Misc

## Analysis

### Scrape

In [ ]:
reader = nytc.OpEdReader()
reader.save_oped_articles_all_pages(begindate='20160101',enddate='20160331')

In [ ]:
reader2 = nytc.OpEdReader()
reader2.save_oped_articles_all_pages(begindate='20161001',enddate='20160107')

In [ ]:
reader3 = nytc.OpEdReader()
reader3.save_oped_articles_all_pages(begindate='20150701',enddate='20151011')

### Load

In [162]:
reader = nytc.OpEdReader.init_from_file('one','one100.csv')

In [178]:
reader2 = nytc.OpEdReader.init_from_file('two','two100.csv')

In [ ]:
reader3 = nytc.OpEdReader.init_from_file('three','three100.csv')

# Text Manipulation

In [6]:
import nltk

In [132]:
PUNCTUATION = """!@#$%^&*()_+=][{}'-";:/?\.,~`"""

def tidy_text(text):
    """ Does the following: 1. Tokenises words 2. Removes punctuation 3. Removes stop words 4. Puts words through the snowball stemmer"""

    stemmer = nltk.stem.snowball.EnglishStemmer()
    stopwords = nltk.corpus.stopwords.words('english')

    outwords = []
    for word in text.split():
        word = word.strip(PUNCTUATION).lower()
        if word not in stopwords:
            if len(word) > 0:
                if not word[0].isdigit():
                    outwords.append(stemmer.stem(word))

    return outwords


## Foreign Language

In [107]:
def locate_foreign(tidiedtext):
    for idx, text in enumerate(tidiedtext):
        if 'de' in text and 'la' in text:
            print('Match')
            print(idx)
            print(text[:20])

In [100]:
text2 = reader2.data['full_text']
tidiedtext2 = [tidy_text(item.decode('utf-8')) for item in text2]

In [104]:
reader2.data['url'].iloc[]

'http://www.nytimes.com/2015/11/17/opinion/les-attentats-a-paris-revelent-les-limites-de-daesh.html'

### Drop Foreign Articles

In [170]:
def drop_rows_all(reader,indices):
    print(reader.data.shape)
    reader.drop_rows(indices)
    print(reader.data.shape)
    csvfilename = '{0}_english{1}.csv'.format(reader.name,100)
    reader.save_to_csv(csvfilename)

### Source #1

In [171]:
drop_rows_all(reader,[105,356,509,535,661,742,762,840,868,913])

(937, 8)
(927, 8)


### Source #2

In [179]:
drop_rows_all(reader2,[27,35,36,58,380,400,508,553,555,802,842,882])

(952, 8)
(940, 8)


### Source #3

In [179]:
drop_rows_all(reader3,[])

(952, 8)
(940, 8)


# Final Result

In [183]:
reader_english = OpEdReader.init_from_file('one_english','one_english100.csv')
reader2_english = OpEdReader.init_from_file('two_english','two_english100.csv')
# reader3_english = OpEdReader.init_from_file('three_english','three_english100.csv')
readers_english = [reader_english,reader2_english]

## Tidy the Text, Join all Datasets

In [141]:
dataall = pd.concat([reader.data for reader in readers_english])
text = dataall['full_text']
tidiedtext = [tidy_text(item.decode('utf-8')) for item in text]
tidiedtextjoined = [' '.join(text) for text in tidiedtext]

In [186]:
dataall = dataall.reset_index(drop=True)

## Distribution of 'Virality'

In [204]:
plt.hist(dataall['share_count'],bins=100)
plt.show()

# LDA

In [41]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from __future__ import print_function
from time import time

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation
from sklearn.datasets import fetch_20newsgroups

In [38]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [148]:
n_features = 1000
n_topics = 10
n_top_words = 20

In [145]:
# Use tf (raw term count) features for LDA.
print("Extracting tf features for LDA...")
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=n_features)
t0 = time()
tf = tf_vectorizer.fit_transform(tidiedtextjoined)
print("done in %0.3fs." % (time() - t0))


Extracting tf features for LDA...
done in 0.972s.


In [149]:
print("Fitting LDA models with tf features, n_samples=%d and n_features=%d..."
      % (n_samples, n_features))
lda = LatentDirichletAllocation(n_topics=n_topics, max_iter=20,
                                learning_method='online', learning_offset=50.,
                                random_state=0)
t0 = time()
lda.fit(tf)
print("done in %0.3fs." % (time() - t0))


Fitting LDA models with tf features, n_samples=2000 and n_features=1000...
done in 30.831s.


In [150]:
print("\nTopics in LDA model:")
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)


Topics in LDA model:
Topic #0:
women school student black educ children colleg univers men abort one famili woman parent percent white class mani marriag work
Topic #1:
gun mr law polic state offic court case peopl prison crime right public year justic new investig crimin would sentenc
Topic #2:
research studi health peopl one year patient like drug percent effect found mani care use also medic doctor risk new
Topic #3:
state mr unit govern countri islam war american nation european refuge presid muslim polit would attack year militari europ one
Topic #4:
isi saudi iraq iran obama syria middl east need bomb defeat want moder order imposs foreign let take problem join
Topic #5:
one peopl it like time said would know go get way us even year make say want day new thing
Topic #6:
new year would tax citi percent state econom chang economi need rate polici one like countri increas hous system climat
Topic #7:
state govern compani would feder law use congress requir bill year water public ch

In [151]:
doc_topic_distrib = lda.transform(tf)

In [191]:
doc_topic_distrib /= doc_topic_distrib.sum(axis=1)[:, np.newaxis]

In [205]:
fig = plt.figure(1)
ax = fig.gca()
ax = ax.matshow(doc_topic_distrib, cmap=plt.cm.gray, aspect = '0.01')
plt.show()